<a href="https://colab.research.google.com/github/victorog17/soulcode_aulas_apache_beam/blob/main/Apache_Beam_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALANDO APACHE BEAM INTERACTIVE**

In [ ]:
pip install apache-beam[interactive]

**BIBLIOTECAS**

In [1]:
import apache_beam as beam

**MAP**

In [14]:
p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/voos_sample.csv', skip_header_lines= 0)
    |'Separador' >> beam.Map(lambda record: record.split(','))
    #|'Saída de dados' >> beam.Map(print)
    #|'Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato.txt')
)
p1.run()

In [ ]:
p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/voos_sample.csv', skip_header_lines= 0)
    |'Separador' >> beam.Map(lambda record: record.split(','))
    |'Filtrar por' >> beam.Filter(lambda record: record[3] == 'JFK')
    #|'Saída de dados' >> beam.Map(print)
    #|'Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultatoJFK.txt')
)
p1.run()

**FLATMAP**

In [12]:
p2 = beam.Pipeline()

poema = (
    p2
    |'Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/poema.txt')
    |'Separador' >> beam.FlatMap(lambda record: record.split(' '))
    #|'Saída de dados' >> beam.FlatMap(print)
    #|'Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_poema.txt')
)
p2.run()

In [15]:
palavras = ['quatro', 'que']

def rastrearPalavras(i):
  if i in palavras:
    return True

p2 = beam.Pipeline()

poema = (
    p2
    |'Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/poema.txt')
    |'Separador' >> beam.FlatMap(lambda record: record.split(' '))
    |'Encontrar as palavras' >> beam.Filter(rastrearPalavras)
    |'Saída de dados' >> beam.FlatMap(print)
    #|'Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_poema_filter.txt')
)
p2.run()

quatro
que
quatro
quatro
que
quatro


**COMBINAÇÃO**

In [19]:
p1 = beam.Pipeline()

voos_atrasados = (
    p1
    |'Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/voos_sample.csv', skip_header_lines= 0)
    |'Separador' >> beam.Map(lambda record: record.split(','))
    |'Aeroportos atrasados' >> beam.Filter(lambda record: int(record[8]) > 0)
    |'Agregação de colunas' >> beam.Map(lambda record: (record[3], int(record[8])))
    |'Quantidade de atrasos' >> beam.combiners.Count.PerKey()
    #|'Combinar os dados' >> beam.CombinePerKey(sum)
    |'Saída de dados' >> beam.Map(print)
    #|'Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_voos_atrasados.txt')
)
p1.run()

('JFK', 5)
('DFW', 2)
('OGG', 1)
('LAX', 2)
('SFO', 2)


**PROCESSAMENTO DE DOIS PIPELINES PARALELAMENTE**

In [27]:
p1 = beam.Pipeline()

atrasados_tempo = (
    p1
    |'1.Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/voos_sample.csv', skip_header_lines= 0)
    |'1.Separador' >> beam.Map(lambda record: record.split(','))
    |'1.Aeroportos atrasados' >> beam.Filter(lambda record: int(record[8]) > 0)
    |'1.Agregação de colunas' >> beam.Map(lambda record: (record[3], int(record[8])))
    |'1.Combinar os dados' >> beam.CombinePerKey(sum)
    #|'Saída de dados' >> beam.Map(print)
    #|'Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_voos_atrasados.txt')
)

atrasos_qtd = (
    p1
    |'2.Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/voos_sample.csv', skip_header_lines= 0)
    |'2.Separador' >> beam.Map(lambda record: record.split(','))
    |'2.Aeroportos atrasados' >> beam.Filter(lambda record: int(record[8]) > 0)
    |'2.Agregação de colunas' >> beam.Map(lambda record: (record[3], int(record[8])))
    |'2.Quantidade de atrasos' >> beam.combiners.Count.PerKey()
    #|'Saída de dados' >> beam.Map(print)
    #|'Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_voos_atrasados.txt')
)

tabela_final = (
    {'Qtd_Atrasos': atrasos_qtd, 'Tempo_Atrasos': atrasados_tempo}
    |'Agrupamento das Pipes' >> beam.CoGroupByKey()
    |'Imprimir resultado' >> beam.Map(print)
    #|'Gravar' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/atrasos_consolidados.txt')
)
p1.run()

('JFK', {'Qtd_Atrasos': [5], 'Tempo_Atrasos': [94]})
('DFW', {'Qtd_Atrasos': [2], 'Tempo_Atrasos': [153]})
('OGG', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [95]})
('LAX', {'Qtd_Atrasos': [2], 'Tempo_Atrasos': [12]})
('SFO', {'Qtd_Atrasos': [2], 'Tempo_Atrasos': [208]})
